In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
from zipfile import ZipFile
from io import BytesIO
import pandas as pd

# Image manipulation.
import PIL.Image
from IPython.display import display

#loading and checking the format of the ground truth csv
train_truth = pd.read_csv('/kaggle/input/conv-labels/converted_file.csv')
train_truth.head()

In [ ]:
train_truth.label.value_counts()

In [ ]:
import os
from PIL import Image
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.data_frame = self.data_frame.dropna(subset=['label'])  # Remove rows with missing labels
        self.root_dir = root_dir
        self.transform = transform
        self.label_map = {'upside_down': 0, 'rotated_left': 1, 'rotated_right': 2, 'upright': 3}  # Define your label mapping

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.label_map[self.data_frame.iloc[idx, 1]]  # Map label to integer using the dictionary

        if self.transform:
            image = self.transform(image)

        return image, label



# Define transformations for data augmentation
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Assuming 3 channels
])

# Define dataset
custom_dataset = CustomDataset(csv_file="/kaggle/input/conv-labels/converted_file.csv",
                               root_dir="/kaggle/input/imageds/myDS",
                               transform=transform)

# Split dataset into train and validation
train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(custom_dataset, [train_size, val_size])

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=False)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self, num_classes=4):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 512)  # Correct size calculation
        self.fc2 = nn.Linear(512, num_classes)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.relu(self.conv2(x))
        x = self.dropout(x)
        x = self.pool(self.relu(self.conv3(x)))
        x = self.relu(self.conv4(x))
        x = self.dropout(x)
        #print(x.size())  # Print the size of the tensor after the fourth convolutional layer
        x = x.view(-1, 64 * 5 * 5)  # Correct reshaping
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
# Load CIFAR-10 dataset
train_dataset = train_dataset
val_dataset = validation_dataset

#train_dataset.dataset.transform = transform_train
#validation_dataset.dataset.transform = transform_val

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

In [ ]:
model = CNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []

# Training the model
num_epochs = 10000
best_val_acc = 0.0  # Variable to keep track of best validation accuracy
best_model_path = None

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    print(f"Epoch {epoch+1}/{num_epochs}")
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        if (i+1) % 10 == 0:  # Print every 10 batches
            avg_loss = running_loss / total
            accuracy = correct / total
            # print(f"Batch [{i+1}/{len(train_loader)}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    # print(f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_acc = val_correct / val_total
    # print(f"Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.4f}")
    train_losses.append(epoch_loss)
    valid_losses.append(val_epoch_loss)
    train_accuracies.append(epoch_acc)
    valid_accuracies.append(val_epoch_acc)
    print(f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.4f}")

    # Save the model if the current validation accuracy is better than the previous best
    if val_epoch_acc > best_val_acc:
        if best_model_path:
            os.remove(best_model_path)  # Remove previously saved best model
        best_val_acc = val_epoch_acc
        best_model_path = f"best_model_{val_epoch_acc:.4f}_valACC_{num_epochs}_epoch.pth"
        torch.save(model.state_dict(), best_model_path)

print(f"Best validation accuracy: {best_val_acc}")

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(train_losses) + 1)

# Plot losses
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'bo-', label='Training loss')
plt.plot(epochs, valid_losses, 'ro-', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot accuracies
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, 'bo-', label='Training accuracy')
plt.plot(epochs, valid_accuracies, 'ro-', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
